In [86]:
cfcs = ['cf74','cf75','cf76','cf77','cf78','cf79']
vocabularios = [{}]

for i in range(0,6):
    voc = {}
    arq = open('cfc/' + cfcs[i])
    documentos = arq.read()
    documentos = documentos.split('\n\n')
    
    for documento in documentos:
        if "" in documento:
            break
        if len(documento) < 5:
            #print('erru')
            continue
        else:
            try:
                autor = documento[documento.index('\nAU')+3:documento.index('\nTI')].split()
            except:
                autor = []
            try:
                title = documento[documento.index('\nTI')+3:documento.index('\nSO')].split()
            except:
                title = []
            try:
                fonte = documento[documento.index('\nSO')+3:documento.index('\nMJ')].split()
            except:
                fonte = []
            try:
                mj = documento[documento.index('\nMJ')+3:documento.index('\nMN')].split()
            except:
                mj = []

            try:
                mn = documento[documento.index('MN')+3:documento.index('\nAB')].split()
            except:
                mn = documento[documento.index('MN')+3:documento.index('\nEX')].split() 


            try:
                try:
                    ab = documento[documento.index('\nAB')+3:documento.index('\nRF')].split()
                except:
                    ab = documento[documento.index('\nAB')+3:documento.index('\nCT')].split()                  
            except:
                try:
                    ab = documento[documento.index('\nEX')+3:documento.index('\nRF')].split()
                except:
                    try:
                        ab = documento[documento.index('\nEX')+3:].split()
                    except:
                        ab = documento[documento.index('\nAB')+3:].split()


            #print(title)
            #print(autor)
            #print(fonte)
            #print(mj)
            #print(mn)
            #print(ab)
            #print(cfcs[i])
        
    for w in title:
        w = w.lower()
        try:
            voc[w][1] += 1
        except:
            voc[w] = [cfcs[i],1]
            
    vocabularios.insert(i,voc)
    
    
vocabularios

[{'hl-a': ['cf74', 1],
  'antigens': ['cf74', 1],
  'in': ['cf74', 1],
  'cystic': ['cf74', 1],
  'fibrosis.': ['cf74', 1]},
 {'isoenzymes': ['cf75', 1],
  'of': ['cf75', 2],
  'alkaline': ['cf75', 1],
  'phosphatase': ['cf75', 1],
  'in': ['cf75', 1],
  'the': ['cf75', 1],
  'serum': ['cf75', 1],
  'patients': ['cf75', 1],
  'with': ['cf75', 1],
  'cystic': ['cf75', 1],
  'fibrosis.': ['cf75', 1]},
 {"crohn's": ['cf76', 1],
  'disease': ['cf76', 1],
  '-': ['cf76', 1],
  'a': ['cf76', 1],
  'cause': ['cf76', 1],
  'of': ['cf76', 1],
  'arthritis,': ['cf76', 1],
  'oxalate': ['cf76', 1],
  'stones': ['cf76', 1],
  'and': ['cf76', 1],
  'fistulae': ['cf76', 1],
  'in': ['cf76', 1],
  'cystic': ['cf76', 1],
  'fibrosis.': ['cf76', 1]},
 {'cystic': ['cf77', 1],
  'fibrosis': ['cf77', 1],
  'in': ['cf77', 1],
  'an': ['cf77', 1],
  'infant': ['cf77', 1],
  'presenting': ['cf77', 1],
  'with': ['cf77', 1],
  'metabolic': ['cf77', 1],
  'alkalosis.': ['cf77', 1]},
 {'mucopolysaccharides': ['

In [91]:
'65 '.isdigit()

False

In [92]:
ok = [title, autor]

In [95]:
for x in ok:
    for nx in x:
        print(nx)

Deficiencies
of
essential
fatty
acids
and
vitamin
E
in
cystic
fibrosis.
Bohles-H.
Heid-H.
Stehr-K.
Fekl-W.
